In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
# movies.head(2)
# credits.head(4)
# credits.head(1)['cast'].values

In [3]:
#merging the datasets
movies = movies.merge(credits, on='title')
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [4]:
'''columns that will help to create a tag for each movie (not including numerical data)
genres, id, title, overview, cast, crew
'''
movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [5]:
#since only 3 values in overview are null we will remove them
movies.dropna(inplace=True)

In [6]:
import ast 
def convert(li):
    res = []
    for i in ast.literal_eval(li):
        res.append(i['name'])
    return res

In [7]:
type(movies['genres'][0])
# the value in the column considers the list as string so we have to make it consider the literal value that is list by using ast.literal_eval()

str

In [8]:
movies['genres'] = movies['genres'].apply(convert)

In [9]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head(1)

In [10]:
# function to get the names of first 4 cast members
def convert3(li):
    count = 0
    res = []
    for i in ast.literal_eval(li):
        if count != 3:
            res.append(i['name'])
        else:
            break
    return res

In [11]:
movies['cast'] = movies['cast'].apply(convert3)

In [12]:
# function to get the names of directors from crew
def getDirectors(li):
    res = []
    for i in ast.literal_eval(li):
        if i['job'] == 'Director':
            res.append(i['name'])
            break
    return res

In [13]:
movies['crew'] = movies['crew'].apply(getDirectors)

In [14]:
#converting overview (string) to list of individual words
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [15]:
#creating a single word for each unique entity
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [ ]:
movies.head(1)

In [16]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']

In [17]:
newMovies = movies[['id', 'title', 'tags']]

In [ ]:
newMovies.head(1)

In [18]:
# converting the tags (list) to string
newMovies['tags'] = newMovies['tags'].apply(lambda x:" ".join(x))

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
newMovies['tags'] = newMovies['tags'].apply(lambda x:x.lower())

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
# stemming to remove redundancies in the tags (activity, activities --> activity)
from nltk.stem import PorterStemmer

ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return(" ".join(y))

In [21]:
newMovies['tags'] = newMovies['tags'].apply(stem)

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
# generate vectors of each tag using sklearn's CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer as cv
obj = cv(max_features=5000, stop_words='english')

In [23]:
vectors = obj.fit_transform(newMovies['tags']).toarray()

In [24]:
# get the 5000 most common words in tags
obj.get_feature_names()

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '007',
 '10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '18th',
 '19',
 '1930',
 '1950',
 '1950s',
 '1960',
 '1960s',
 '1970',
 '1970s',
 '1980',
 '1985',
 '1990',
 '19th',
 '20',
 '200',
 '2009',
 '20th',
 '24',
 '25',
 '30',
 '300',
 '3d',
 '40',
 '50',
 '500',
 '50cent',
 '60',
 '70',
 'aaron',
 'aaroneckhart',
 'aaronpaul',
 'aarontaylor',
 'aaronyoo',
 'aasifmandvi',
 'abandon',
 'abbiecornish',
 'abduct',
 'abigailbreslin',
 'abil',
 'abl',
 'aboard',
 'abov',
 'abrahambenrubi',
 'abus',
 'academi',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'account',
 'accus',
 'ace',
 'achiev',
 'act',
 'action',
 'activ',
 'activist',
 'activities',
 'actor',
 'actress',
 'actual',
 'adam',
 'adambrodi',
 'adamgodley',
 'adamgoldberg',
 'adamlefevr',
 'adammckay',
 'adamsandl',
 'adamscott',
 'adamshankman',
 'adapt',
 'add',
 'addict',
 'adewaleakinnuoye',
 'adjust',
 'admir',
 'admit',
 'adolesc',
 'adopt',
 'ador',

In [25]:
# calculating distance between the vectors using cosine_similarity of sklearn
from sklearn.metrics.pairwise import cosine_similarity as cs
dist = cs(vectors)

In [45]:
# get top 5 recommendations when a movie title is given
def recommend(movie):
    ind = newMovies[newMovies['title'] == movie].index[0]
    '''get the dist vector of the given movie -> enumerate them (give index to all the values so we dont loose them after sorting) 
    -> sort the distances based on the second value in desc order -> take the first six from second position.
    '''
    top5 = sorted(enumerate(dist[ind]), reverse=True, key=lambda x:x[1])[1:6]
    
    for i in top5:
        print(newMovies.iloc[i[0]].title)
#         print(i[0])

In [51]:
recommend('Batman Begins')

The Dark Knight
Synecdoche, New York
Sexy Beast
Copying Beethoven
Rockaway
